In [13]:
import os
import json
import shutil
from pathlib import Path

# Define paths (compatible with Jupyter notebooks)
# Giả sử notebook này nằm trong notebooks/ folder
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir

DATA_FOLDER = PROJECT_ROOT / 'notebooks' / 'submission_final' / '23127011'  # Folder chứa preds.json
DATA2_FOLDER = PROJECT_ROOT / '..' / 'data_output_v2'  # Folder chứa các file còn lại


In [14]:
# Kiểm tra đường dẫn
print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"DATA_FOLDER: {DATA_FOLDER}")
print(f"DATA2_FOLDER: {DATA2_FOLDER}")
print(f"\nDATA_FOLDER exists: {DATA_FOLDER.exists()}")
print(f"DATA2_FOLDER exists: {DATA2_FOLDER.exists()}")


PROJECT_ROOT: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011
DATA_FOLDER: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011
DATA2_FOLDER: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\..\data_output_v2

DATA_FOLDER exists: True
DATA2_FOLDER exists: True


# Khám phá cấu trúc thư mục
Xem cấu trúc của folder `data` và `data2`

In [15]:
def explore_folder_structure(folder_path, max_depth=3, current_depth=0):
    """Hiển thị cấu trúc thư mục"""
    if not folder_path.exists():
        print(f"Folder không tồn tại: {folder_path}")
        return
    
    indent = "  " * current_depth
    print(f"{indent}{folder_path.name}/")
    
    if current_depth >= max_depth:
        return
    
    try:
        items = sorted(folder_path.iterdir())
        for item in items[:10]:  # Giới hạn 10 items đầu tiên
            if item.is_dir():
                explore_folder_structure(item, max_depth, current_depth + 1)
            else:
                print(f"{indent}  {item.name}")
        
        if len(items) > 10:
            print(f"{indent}  ... ({len(items) - 10} items nữa)")
    except PermissionError:
        print(f"{indent}  [Permission denied]")

print("=== Cấu trúc DATA_FOLDER ===")
explore_folder_structure(DATA_FOLDER)
print("\n=== Cấu trúc DATA2_FOLDER ===")
explore_folder_structure(DATA2_FOLDER)


=== Cấu trúc DATA_FOLDER ===
23127011/
  2403-00530/
    pred.json
  2403-00531/
    pred.json
  2403-00533/
    pred.json
  2403-00537/
    pred.json
  2403-00540/
    pred.json
  2403-00545/
    pred.json
  2403-00548/
    pred.json
  2403-00549/
    pred.json
  2403-00550/
    pred.json
  2403-00552/
    pred.json
  ... (1018 items nữa)

=== Cấu trúc DATA2_FOLDER ===
data_output_v2/
  2403-00530/
    hierarchy.json
    labels.json
    metadata.json
    references.json
    refs.bib
  2403-00531/
    hierarchy.json
    labels.json
    metadata.json
    references.json
    refs.bib
  2403-00532/
    hierarchy.json
    labels.json
    metadata.json
    references.json
    refs.bib
  2403-00533/
    hierarchy.json
    labels.json
    metadata.json
    references.json
    refs.bib
  2403-00534/
    hierarchy.json
    metadata.json
    references.json
    refs.bib
  2403-00535/
    hierarchy.json
    metadata.json
    references.json
    refs.bib
  2403-00536/
    hierarchy.json
    labels

# Đọc dữ liệu từ các folder
Đọc tất cả các file `preds.json` từ folder `data` và các file khác từ folder `data2`

In [16]:
def get_all_preds_files(data_folder):
    """Tìm tất cả các file preds.json trong cấu trúc data/paper_id/pred.json"""
    preds_files = []
    
    if not data_folder.exists():
        print(f"Folder {data_folder} không tồn tại!")
        return preds_files
    
    # Duyệt qua từng paper_id
    for paper_folder in data_folder.iterdir():
        if not paper_folder.is_dir():
            continue
            
        # Tìm file pred.json
        preds_file = paper_folder / 'pred.json'
        if preds_file.exists():
            preds_files.append({
                'paper_id': paper_folder.name,
                'preds_path': preds_file
            })
    
    return preds_files

def get_valid_paper_ids(data_folder):
    """Lấy danh sách các paper_id có trong data folder"""
    paper_ids = set()
    
    if not data_folder.exists():
        print(f"Folder {data_folder} không tồn tại!")
        return paper_ids
    
    for paper_folder in data_folder.iterdir():
        if paper_folder.is_dir():
            paper_ids.add(paper_folder.name)
    
    return paper_ids

# Tìm tất cả các file preds.json
preds_files = get_all_preds_files(DATA_FOLDER)
print(f"Tìm thấy {len(preds_files)} file pred.json")
if preds_files:
    print(f"\nVí dụ: {preds_files[0]}")

# Lấy danh sách paper_id hợp lệ
valid_paper_ids = get_valid_paper_ids(DATA_FOLDER)
print(f"\nCó {len(valid_paper_ids)} paper_id trong DATA_FOLDER")
print(f"Paper IDs (5 đầu tiên): {sorted(list(valid_paper_ids))[:5]}")


Tìm thấy 1028 file pred.json

Ví dụ: {'paper_id': '2403-00530', 'preds_path': WindowsPath('d:/Coding/School/Y3-K1/Intro2DS/DS - LAB 2/Milestone2_Project/23127011/notebooks/submission_final/23127011/2403-00530/pred.json')}

Có 1028 paper_id trong DATA_FOLDER
Paper IDs (5 đầu tiên): ['2403-00530', '2403-00531', '2403-00533', '2403-00537', '2403-00540']


In [18]:
def get_all_data2_files(data2_folder, valid_paper_ids=None):
    """
    Tìm tất cả các file trong data2/paper_id/
    
    Args:
        data2_folder: Đường dẫn đến folder data2
        valid_paper_ids: Set các paper_id hợp lệ. Nếu None, lấy tất cả.
    """
    data2_files = []
    
    if not data2_folder.exists():
        print(f"Folder {data2_folder} không tồn tại!")
        return data2_files
    
    # Duyệt qua từng paper_id
    for paper_folder in data2_folder.iterdir():
        if not paper_folder.is_dir():
            continue
        
        # Kiểm tra paper_id có hợp lệ không
        if valid_paper_ids is not None and paper_folder.name not in valid_paper_ids:
            continue  # Bỏ qua paper_id không có trong DATA_FOLDER
            
        # Lấy tất cả các file trong paper_folder, LOẠI TRỪ labels.json
        files = [f for f in paper_folder.iterdir() 
                 if f.is_file() and f.name != 'labels.json']
        if files:
            data2_files.append({
                'paper_id': paper_folder.name,
                'files': files,
                'source_folder': paper_folder
            })
    
    return data2_files

# Tìm tất cả các file từ data2 (chỉ lấy paper_id hợp lệ)
data2_files = get_all_data2_files(DATA2_FOLDER, valid_paper_ids)
print(f"\nTìm thấy {len(data2_files)} folders trong data2 (chỉ các paper_id hợp lệ)")
if data2_files:
    print(f"\nVí dụ folder đầu tiên:")
    print(f"  Paper ID: {data2_files[0]['paper_id']}")
    print(f"  Số file: {len(data2_files[0]['files'])}")
    print(f"  Các file: {[f.name for f in data2_files[0]['files']]}")



Tìm thấy 1027 folders trong data2 (chỉ các paper_id hợp lệ)

Ví dụ folder đầu tiên:
  Paper ID: 2403-00530
  Số file: 4
  Các file: ['hierarchy.json', 'metadata.json', 'references.json', 'refs.bib']


# Gộp dữ liệu từ data2 vào data
Copy các file từ `data2/student_id/paper_id/` vào `data/student_id/paper_id/`

In [19]:
def merge_data2_into_data(data_folder, data2_files, dry_run=True):
    """
    Copy các file từ data2 vào data
    
    Args:
        data_folder: Đường dẫn đến folder data
        data2_files: Danh sách các folder từ data2
        dry_run: Nếu True, chỉ hiển thị preview mà không thực sự copy
    """
    copied_count = 0
    skipped_count = 0
    
    for item in data2_files:
        paper_id = item['paper_id']
        source_folder = item['source_folder']
        
        # Tạo đường dẫn đích
        dest_folder = data_folder / paper_id
        
        # Kiểm tra xem folder đích có tồn tại không
        if not dest_folder.exists():
            print(f"⚠️  Bỏ qua {paper_id} - folder đích không tồn tại")
            skipped_count += 1
            continue
        
        # Copy từng file
        for source_file in item['files']:
            dest_file = dest_folder / source_file.name
            
            if dry_run:
                print(f"📋 Preview: {source_file.name} -> {paper_id}/")
            else:
                try:
                    shutil.copy2(source_file, dest_file)
                    print(f"✅ Copied: {source_file.name} to {paper_id}/")
                    copied_count += 1
                except Exception as e:
                    print(f"❌ Error copying {source_file.name}: {e}")
    
    if dry_run:
        total_files = len([f for item in data2_files for f in item['files']])
        print(f"\n📊 Preview: Sẽ copy {total_files} files vào {len(data2_files)} folders")
        print(f"   Bỏ qua {skipped_count} folders không tồn tại")
    else:
        print(f"\n✅ Đã copy {copied_count} files")
        print(f"⚠️  Bỏ qua {skipped_count} folders")
    
    return copied_count, skipped_count

# Preview trước khi thực sự copy
print("=== PREVIEW MODE ===")
merge_data2_into_data(DATA_FOLDER, data2_files, dry_run=True)


=== PREVIEW MODE ===
📋 Preview: hierarchy.json -> 2403-00530/
📋 Preview: metadata.json -> 2403-00530/
📋 Preview: references.json -> 2403-00530/
📋 Preview: refs.bib -> 2403-00530/
📋 Preview: hierarchy.json -> 2403-00531/
📋 Preview: metadata.json -> 2403-00531/
📋 Preview: references.json -> 2403-00531/
📋 Preview: refs.bib -> 2403-00531/
📋 Preview: hierarchy.json -> 2403-00533/
📋 Preview: metadata.json -> 2403-00533/
📋 Preview: references.json -> 2403-00533/
📋 Preview: refs.bib -> 2403-00533/
📋 Preview: hierarchy.json -> 2403-00537/
📋 Preview: metadata.json -> 2403-00537/
📋 Preview: references.json -> 2403-00537/
📋 Preview: refs.bib -> 2403-00537/
📋 Preview: hierarchy.json -> 2403-00540/
📋 Preview: metadata.json -> 2403-00540/
📋 Preview: references.json -> 2403-00540/
📋 Preview: refs.bib -> 2403-00540/
📋 Preview: hierarchy.json -> 2403-00545/
📋 Preview: metadata.json -> 2403-00545/
📋 Preview: references.json -> 2403-00545/
📋 Preview: refs.bib -> 2403-00545/
📋 Preview: hierarchy.json -> 24

(0, 0)

# Thực hiện copy thật
Sau khi xác nhận preview, chạy lệnh dưới đây để thực sự copy các file

In [20]:
# Thực hiện copy (bỏ comment để chạy)
merge_data2_into_data(DATA_FOLDER, data2_files, dry_run=False)


✅ Copied: hierarchy.json to 2403-00530/
✅ Copied: metadata.json to 2403-00530/
✅ Copied: references.json to 2403-00530/
✅ Copied: refs.bib to 2403-00530/
✅ Copied: hierarchy.json to 2403-00531/
✅ Copied: metadata.json to 2403-00531/
✅ Copied: references.json to 2403-00531/
✅ Copied: refs.bib to 2403-00531/
✅ Copied: hierarchy.json to 2403-00533/
✅ Copied: metadata.json to 2403-00533/
✅ Copied: references.json to 2403-00533/
✅ Copied: refs.bib to 2403-00533/
✅ Copied: hierarchy.json to 2403-00537/
✅ Copied: metadata.json to 2403-00537/
✅ Copied: references.json to 2403-00537/
✅ Copied: refs.bib to 2403-00537/
✅ Copied: hierarchy.json to 2403-00540/
✅ Copied: metadata.json to 2403-00540/
✅ Copied: references.json to 2403-00540/
✅ Copied: refs.bib to 2403-00540/
✅ Copied: hierarchy.json to 2403-00545/
✅ Copied: metadata.json to 2403-00545/
✅ Copied: references.json to 2403-00545/
✅ Copied: refs.bib to 2403-00545/
✅ Copied: hierarchy.json to 2403-00548/
✅ Copied: metadata.json to 2403-0054

(4108, 0)